In [1]:
# -- driver_run_forecast_LV1.py  --
# master python script to do a full LV1 forecast simulation

import sys
import pickle
import numpy as np
import os
from datetime import datetime, timedelta
import gc
import resource
import subprocess

##############

sys.path.append('../sdpm_py_util')

import atm_functions as atmfuns
import ocn_functions as ocnfuns
import grid_functions as grdfuns
import plotting_functions as pltfuns
from get_PFM_info import get_PFM_info
import init_funs as infuns
from make_LV1_dotin_and_SLURM import make_LV1_dotin_and_SLURM
from run_slurm_LV1 import run_slurm_LV1

print('\nStarting the LV1 simulation, Current time ', datetime.now())

# we are going to make a forecast
run_type = 'forecast'

# PFM has all of the information needed to run the model
#clean_start = True
#initialize_simulation(clean_start)
PFM=get_PFM_info()



Starting the LV1 simulation, Current time  2024-09-18 16:37:15.061982


/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_functions.py:24: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


In [3]:
# return the hycom forecast date based on the PFM simulation times
yyyymmdd_hy = infuns.determine_hycom_foretime() 
t1  = PFM['fetch_time']                              # this is the first time of the PFM forecast
# now a string of the time to start ROMS (and the 1st atm time too)
yyyymmddhhmm_pfm = "%d%02d%02d%02d%02d" % (t1.year, t1.month, t1.day, t1.hour, t1.minute)
t2  = t1 + PFM['forecast_days'] * timedelta(days=1)  # this is the last time of the PFM forecast

print('going to do a PFM forecast from')
print(t1)
print('to')
print(t2)
print('\n')

# now make the catted hycom data .nc file
#ocnfuns.cat_hycom_to_onenc(yyyymmdd_hy,[t1,t2])
print('we will use the hycom forecast from')
print(yyyymmdd_hy)
print('for the PFM forecast.\n')

print('catting the hycom.nc files into one...')
ocnfuns.cat_hycom_to_onenc(yyyymmdd_hy,[t1,t2])
print('...done.\n')

print('making the hycom pickle files from catted hycom.nc...')
ocnfuns.hycom_cat_to_pickles(yyyymmdd_hy)
print('...done.\n')



making a PFM simulation starting from
2024-09-18 12:00:00
and ending at
2024-09-21 00:00:00
getting the right hycom data...
/scratch/PFM_Simulations/hycom_data/hy_s3z_2024-09-17T12:00_2024-09-19T03:00.nc
/scratch/PFM_Simulations/hycom_data/hy_t3z_2024-09-17T12:00_2024-09-19T03:00.nc
/scratch/PFM_Simulations/hycom_data/hy_u3z_2024-09-17T12:00_2024-09-19T03:00.nc
/scratch/PFM_Simulations/hycom_data/hy_v3z_2024-09-17T12:00_2024-09-19T03:00.nc
/scratch/PFM_Simulations/hycom_data/hy_s3z_2024-09-17T12:00_2024-09-19T06:00.nc
/scratch/PFM_Simulations/hycom_data/hy_t3z_2024-09-17T12:00_2024-09-19T06:00.nc
/scratch/PFM_Simulations/hycom_data/hy_u3z_2024-09-17T12:00_2024-09-19T06:00.nc
/scratch/PFM_Simulations/hycom_data/hy_v3z_2024-09-17T12:00_2024-09-19T06:00.nc
/scratch/PFM_Simulations/hycom_data/hy_s3z_2024-09-17T12:00_2024-09-19T09:00.nc
/scratch/PFM_Simulations/hycom_data/hy_t3z_2024-09-17T12:00_2024-09-19T09:00.nc
/scratch/PFM_Simulations/hycom_data/hy_u3z_2024-09-17T12:00_2024-09-19T09:00

oc_open: server error retrieving url: code=3 message="Invalid Parameter Exception: DArrayDimension.setProjection: Bad Projection Request: stop >= size: 33:13"nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco_get_vara()
nco_err_exit(): ERROR Error code is -77. Translation into English with nc_strerror(-77) is "NetCDF: Access failure"


ncks: ERROR User-specified coordinate value range 318 <= time <= 318.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 321 <= time <= 321.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 324 <= time <= 324.5 does not fall within valid coordinate range 1.15889e-310 <= time <= 1.66668e-94
ncks: ERROR User-specified coordinate value range 327 <= time <= 327.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 330 <= time <= 330.5 does not fall within valid coordinate range 1.15889e-310 <= time <= 1.66668e-94
ncks: ERROR User-specified coordinate value range 333 <= time <= 333.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 336 <= time <= 336.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified c

20240916


In [ ]:


fetch_time = PFM['fetch_time'] # the start of the PFM forecast
hy_time = datetime(fetch_time.year,fetch_time.month,fetch_time.day)
hy_time = hy_time - timedelta(days=1)
yyyymmdd = "%d%02d%02d" % (hy_time.year, hy_time.month, hy_time.day)

print(PFM['forecast_days'])

t1  = fetch_time                                     # this is the first time of the PFM forecast
t2  = t1 + PFM['forecast_days'] * timedelta(days=1)  # this is the last time of the PFM forecast

print('making a PFM simulation starting from')
print(t1)
print('and ending at')
print(t2)
print('getting the right hycom data...')
# make stuff below this into a function...
# n0 is the number of files we should have
# num_missing is the number of files we are missing
n0, num_missing = ocnfuns.check_hycom_data(yyyymmdd,[t1,t2])
print('there should be ' + str(n0) + ' files.')
print('there are ' + str(num_missing) + ' missing files.')



In [10]:


if num_missing == 0:
    print('we have the hycom files from the ' + yyyymmdd )
    print('hycom forecast to run the PFM forecast')
    ocnfuns.cat_hycom_to_onenc(yyyymmdd,[t1,t2])
    # cat hycom files to one .nc.
else:
    # try downloading all of the data again
    print('we did not have the files from the ' + yyyymmdd + ' hycom forecast')
    print('but we will try getting that data again. Maybe hycom filled in their .nc files?')
    ocnfuns.get_hycom_data(yyyymmdd)
    n02, num_missing2 = ocnfuns.check_hycom_data(yyyymmdd,[t1,t2])
    if num_missing2 == 0:
        print('Hycom now has the data we need to run the PFM forecast')
        print('catting the hycom.nc files into one...')
        ocnfuns.cat_hycom_to_onenc(yyyymmdd,[t1,t2])
        print('...done')
    else:
        print('we need the change the date of the hycom forecast (but not change the dates of the PFM forecast)')
        # yyyymmdd = yyyymmdd - 1 day
        # do this over again!



we did not have the files from the 20240917hycom forecast
but we will try getting that data again. Maybe hycom filled in their .nc files?


ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"
ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not sear

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"
ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not sear

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

...

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

oc_open: server error retrieving url: code=3 message="Invalid Parameter Exception: DArrayDimension.setProjection: Bad Projection Request: stop >= size: 19:13"nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco_get_vara()
nco_err_exit(): ERROR Error code is -77. Translation into English with nc_strerror(-77) is "NetCDF: Access failure"


ncks: ERROR User-specified coordinate value range 282 <= time <= 282.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 285 <= time <= 285.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 294 <= time <= 294.5 does not fall within valid coordinate range 192 <= time <= 228
ERROR: nco_get_vara() failed to nc_get_vara() variable "water_u"
ERROR NC_EACCESS Access failure
HINT: NC_EACCESS errors signify a problem receiving data from a DAP server. This can occur in multiple ways, including 1) When NCO requests (with nco_var_get()) more data than the server is configured to dispense at one time. A workaround for this might be to request data in smaller amounts at one time. This can be accomplished by accessing hyperslabs or multi-slabs of data as described at http://nco.sf.net/nco.html#mlt. 2) When NCO requests data from a buggy DAP library. The DAP software built i

oc_open: server error retrieving url: code=3 message="Invalid Parameter Exception: DArrayDimension.setProjection: Bad Projection Request: stop >= size: 32:13"nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco_get_vara()
nco_err_exit(): ERROR Error code is -77. Translation into English with nc_strerror(-77) is "NetCDF: Access failure"


ncks: ERROR User-specified coordinate value range 300 <= time <= 300.5 does not fall within valid coordinate range 1.66006e-321 <= time <= 4.63444e-310
ncks: ERROR User-specified coordinate value range 324 <= time <= 324.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 306 <= time <= 306.5 does not fall within valid coordinate range 1.58101e-321 <= time <= 4.63444e-310
ncks: ERROR nco_lmt_evl_dmn_crd() reports domain 3.420000000e+02 <= time <= 3.425000000e+02 brackets no coordinate values
ncks: INFO user-specified coordinate minimum: "2024-09-23T18:00"
ncks: INFO user-specified coordinate maximum: "2024-09-23T18:30"
ncks: ERROR User-specified coordinate value range 333 <= time <= 333.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 330 <= time <= 330.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate valu

In [3]:
ocnfuns.cat_hycom_to_onenc(yyyymmdd,[t1,t2])


In [6]:
7 % 6

1

In [5]:

ocnfuns.get_hycom_data('20240917')


ncks: ERROR User-specified coordinate value range 234 <= time <= 234.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 237 <= time <= 237.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 231 <= time <= 231.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 240 <= time <= 240.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 243 <= time <= 243.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 246 <= time <= 246.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 249 <= time <= 249.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 252 <= time 

oc_open: server error retrieving url: code=3 message="Invalid Parameter Exception: DArrayDimension.setProjection: Bad Projection Request: stop >= size: 32:13"nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco_get_vara()
nco_err_exit(): ERROR Error code is -77. Translation into English with nc_strerror(-77) is "NetCDF: Access failure"


ncks: ERROR User-specified coordinate value range 312 <= time <= 312.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 306 <= time <= 306.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR nco_lmt_evl_dmn_crd() reports domain 3.090000000e+02 <= time <= 3.095000000e+02 brackets no coordinate values
ncks: INFO user-specified coordinate minimum: "2024-09-22T09:00"
ncks: INFO user-specified coordinate maximum: "2024-09-22T09:30"
ncks: ERROR User-specified coordinate value range 315 <= time <= 315.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 321 <= time <= 321.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 318 <= time <= 318.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR nco_lmt_evl_dmn_crd() reports domain 3.240000000e+02 <= time <= 3.

In [6]:
yyyymmdd='20240917'
yyyy = yyyymmdd[0:4]
mm = yyyymmdd[4:6]
dd = yyyymmdd[6:8]
dstr0 = yyyy + '-' + mm + '-' + dd + 'T12:00' # this is the forecast time

t1 =  datetime.strptime(dstr0,"%Y-%m-%dT%H:%M")
t2 = t1 + 8.0 * timedelta(days=1)
times = [t1,t2]
n0, num_missing = ocnfuns.check_hycom_data(yyyymmdd,times)

print('there should be')
print(str(n0))
print('files.')
print('The number of missing files is:')
print(str(num_missing))
print('(0 means we have all files we checked for)')

/scratch/PFM_Simulations/hycom_data/hy_s3z_2024-09-17T12:00_2024-09-19T03:00.nc
/scratch/PFM_Simulations/hycom_data/hy_t3z_2024-09-17T12:00_2024-09-19T03:00.nc
/scratch/PFM_Simulations/hycom_data/hy_u3z_2024-09-17T12:00_2024-09-19T03:00.nc
/scratch/PFM_Simulations/hycom_data/hy_v3z_2024-09-17T12:00_2024-09-19T03:00.nc
/scratch/PFM_Simulations/hycom_data/hy_s3z_2024-09-17T12:00_2024-09-19T06:00.nc
/scratch/PFM_Simulations/hycom_data/hy_t3z_2024-09-17T12:00_2024-09-19T06:00.nc
/scratch/PFM_Simulations/hycom_data/hy_u3z_2024-09-17T12:00_2024-09-19T06:00.nc
/scratch/PFM_Simulations/hycom_data/hy_v3z_2024-09-17T12:00_2024-09-19T06:00.nc
/scratch/PFM_Simulations/hycom_data/hy_s3z_2024-09-17T12:00_2024-09-19T09:00.nc
/scratch/PFM_Simulations/hycom_data/hy_t3z_2024-09-17T12:00_2024-09-19T09:00.nc
/scratch/PFM_Simulations/hycom_data/hy_u3z_2024-09-17T12:00_2024-09-19T09:00.nc
/scratch/PFM_Simulations/hycom_data/hy_s3z_2024-09-17T12:00_2024-09-19T12:00.nc
/scratch/PFM_Simulations/hycom_data/hy_t